In [ ]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials


In [ ]:
# link = 'https://drive.google.com/file/d/1Oe82Rm0lOnRdf_BuK4kV-SYHYc1RPb7Y/view?usp=drive_link' # The shareable link

In [ ]:
# fluff, id = link.split('=')
# print (id) # Verify that you have everything after '='

drive_link


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00


In [ ]:

# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [ ]:

# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import pandas as pd
import os
os.chdir("/content/drive/My Drive/6220_datasets/")
!ls
df_llama=pd.read_csv("/content/drive/My Drive/6220_datasets/llama_1200.csv")
df_gpt2=pd.read_csv("/content/drive/My Drive/6220_datasets/ggpalm_1200.csv")
df_gptneo=pd.read_csv("/content/drive/My Drive/6220_datasets/gptneo_1200.csv")
print(df_llama.shape)
print(df_gptneo.shape)
print(df_gpt2.shape)
assert df_llama.shape == df_gpt2.shape
assert df_llama.shape == df_gptneo.shape
# print(df_llama.head())
min_result = []
llm_list = ["llama", "ggpalm", "gptne"]
for i in range(len(df_llama)):
  score_list = [df_llama.iloc[i,2], df_gpt2.iloc[i,2], df_gptneo.iloc[i,2]]
  min_index = score_list.index(min(score_list))
  current_result = [df_llama.iloc[i,1], llm_list[min_index]]
  min_result.append(current_result)
df = pd.DataFrame(min_result, columns = ['text', 'label'])
print(df.head())
print(df.shape)

# split_ratio = 0.8
# from datasets import Dataset
# tds = Dataset.from_pandas(df.head(int(split_ratio*df.shape[0])))
# vds = Dataset.from_pandas(df.tail(int((1-split_ratio)*df.shape[0]+1)))
# print(tds.shape)
# print(vds.shape)

# from datasets import DatasetDict

# ds = DatasetDict()

# ds['train'] = tds
# ds['validation'] = vds


ggpalm_1200.csv  GPTNeo_100.csv   gptneo_1200.gsheet  llama_1200.csv
GPT2_100.csv	 gptneo_1200.csv  llama_100.csv       my_awesome_model
(1199, 3)
(1199, 3)
(1199, 3)
                                                text   label
0  Actually, to be more accurate, I see things li...  ggpalm
1  "Get your head out of your--... let’s be polit...   gptne
2  In fact, Bynum allegedly responded to Eley on ...   gptne
3  The defendant stayed fully clothed and used a ...   llama
4  The feminist berated Benjamin before an audien...   gptne
(1199, 2)


In [ ]:
my_dict = {
    'llama':'a',
    'ggpalm':'b',
    'gptne':'c'}

def update_cat(x):
    return my_dict[x]

df['label'] = df['label'].apply(lambda x: update_cat(x))

encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

df['ENCODE_CAT'] = df['label'].apply(lambda x: encode_cat(x))
print(df.head())


                                                text label  ENCODE_CAT
0  Actually, to be more accurate, I see things li...     b           0
1  "Get your head out of your--... let’s be polit...     c           1
2  In fact, Bynum allegedly responded to Eley on ...     c           1
3  The defendant stayed fully clothed and used a ...     a           2
4  The feminist berated Benjamin before an audien...     c           1


In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')

In [ ]:
class Triage(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        title = str(self.data.text[index])
        # title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.ENCODE_CAT[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = Triage(train_dataset, tokenizer, MAX_LEN)
testing_set = Triage(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1199, 3)
TRAIN Dataset: (959, 3)
TEST Dataset: (240, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


In [ ]:
class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output


In [ ]:

model = DistillBERTClass()
model.to(device)

DistillBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(i

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accu(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Training Loss per 5000 steps: 1.428366780281067
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 0: 33.785192909280504
Training Loss Epoch: 1.1544557506839435
Training Accuracy Epoch: 33.785192909280504
Training Loss per 5000 steps: 1.0958583354949951
Training Accuracy per 5000 steps: 0.0
The Total Accuracy for Epoch 1: 41.91866527632951
Training Loss Epoch: 1.0898808367550372
Training Accuracy Epoch: 41.91866527632951
Training Loss per 5000 steps: 1.0180730819702148
Training Accuracy per 5000 steps: 50.0
The Total Accuracy for Epoch 2: 47.23670490093848
Training Loss Epoch: 1.040061698978146
Training Accuracy Epoch: 47.23670490093848
Training Loss per 5000 steps: 0.9179469347000122
Training Accuracy per 5000 steps: 50.0
The Total Accuracy for Epoch 3: 55.683003128258605
Training Loss Epoch: 0.9478945033003886
Training Accuracy Epoch: 55.683003128258605
Training Loss per 5000 steps: 0.5553097724914551
Training Accuracy per 5000 steps: 100.0
The Total Accuracy for Epoc

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss = 0; nb_tr_steps = 0; nb_tr_examples = 0
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accu(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [ ]:
print('This is the validation section to print the accuracy and see how it performs')
print('Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch')

acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)


This is the validation section to print the accuracy and see how it performs
Here we are leveraging on the dataloader crearted for the validation dataset, the approcah is using more of pytorch
Validation Loss per 100 steps: 1.4881205558776855
Validation Accuracy per 100 steps: 0.0
Validation Loss Epoch: 1.2461074903607368
Validation Accuracy Epoch: 38.333333333333336
Accuracy on test data = 38.33%


In [ ]:
pip install transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
ds["train"][0]

{'Prompt': 'Corruption involving the contractors is the chief culprit for the prison\'s problems, according to a recent report by the U.S. Government Accountability Office.\nThe GAO report, released in May, said the prison\'s problems are "not unique to the federal prison system."\nThe report said the prison\'s problems are "not unique to the federal prison system."\nThe GAO said the prison\'s problems are "not unique to the federal prison system."',
 'BestLLM': 'llama'}

In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
! pip install -U accelerate
! pip install -U transformers

In [ ]:
accuracy = evaluate.load("accuracy")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# def preprocess_function(examples):

#     return tokenizer(examples["text"], padding="max_length", truncation=True)
max_input_length = 512
def preprocess_function(examples):
    model_inputs = tokenizer(
        text = examples["text"],
        max_length=max_input_length,
        padding='max_length',
        truncation=True,
    )
    labels = tokenizer(examples["label"],truncation=True, padding='max_length')
    model_inputs["label"] = labels["input_ids"]
    return model_inputs

tokenized_ds = ds.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer,padding='max_length')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
id2label = {0: "llama", 1: "gpt20", 2: "gptne"}
label2id = {"llama": 0, "gpt20": 1, "gptne": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            token_type_ids=inputs['token_type_ids']
        )
        loss = th.nn.BCEWithLogitsLoss()(outputs['logits'],
                                          inputs['labels'])
        return (loss, outputs) if return_outputs else loss
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: ignored